## Data Minimisation Experiment

To prepare the data for the experiment, download from the landingzone the detections and images in the following structure:

- `input_folder /`
  - `images`
  - `detections`

Run this notebook to generate the images for the different scenarios which will be saved in `output_folder`. 

In [ ]:
import os
import pandas as pd

from objectherkenning_openbare_ruimte.data_minimisation.data_minimisation import DataMinimisation

# Set appropriate paths
input_folder = "../datasets/oor/data-minimalisation/prep_data"
images_folder = os.path.join(input_folder, "images")
detections_folder =  os.path.join(input_folder, "detections")

output_folder = "../datasets/oor/data-minimalisation/blur_test"

# Folder to store detections in YOLO format
annotations_folder = os.path.join(input_folder, "labels")

In [ ]:
# Convert detections from "landingzone format" back to YOLO annotation files

df = pd.concat([pd.read_csv(os.path.join(detections_folder, file)) for file in os.listdir(detections_folder) if file.endswith(".csv")])
images = [file for file in os.listdir(images_folder) if file.endswith(".jpg")]
df = df[df["image_name"].isin(images)]

os.makedirs(annotations_folder, exist_ok=True)

for image in images:
    out_file = os.path.join(annotations_folder, f"{os.path.splitext(image)[0]}.txt")
    image_df = df[df["image_name"] == image].set_index("image_name")
    image_df.to_csv(out_file, sep=" ", index=False, header=False)

In [ ]:
# Generate images for scenarios

data_minimisation = DataMinimisation()

data_minimisation.process_folder(images_folder, annotations_folder, output_folder, image_format="jpg")